# Task 1 d, Calculating strength reserve factor of all elements

## Please enter your name (daniel, fabian, felix, yannis)

In [1]:
name = "yannis"

## Imports 

In [2]:
import pandas as pd
import sys 
import os

sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *


In [3]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

ValueError: invalid literal for int() with base 10: '2 # DONT CHANGE THIS'

## File Selection 

In [ ]:
#Import the stress of the panels (OLD VERSION)
# paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)

In [ ]:
# This will become the new import for the panel stresses
paneldf_v2 = pd.read_csv(f'../data/{name}/panel_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

# Rename the element column to match
#element_component_df.rename(columns={'Element ID': 'element_id'}, inplace=True)
paneldf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)

# now add the component column to the paneldf_v2 by matching the element_id
paneldf_v2 = add_component_names_to_elements(paneldf_v2, element_component_df)

paneldf_v2.rename(columns={'Loadcase': 'Load Case', 'XX': 'sigmaXX', 'YY': 'sigmaYY', 'XY': 'sigmaXY', 'ZZ': 'sigmaZZ', 'YZ': 'sigmaYZ', 'ZX': 'sigmaXZ'}, inplace=True)
paneldf_v2.drop(columns=['FileID', 'Step', 'Layer'], inplace=True)
paneldf_v2.reset_index(drop=True, inplace=True)
paneldf_v2.set_index('Element ID', inplace=True) # @yannisendr this is possible, since pandas allows non-unique indices, however operations that require an unique index won't work 

# Rename the load case column to match the old one
paneldf_v2['Load Case'] = paneldf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

# copy the new df to the previous one
paneldf = paneldf_v2
#print(paneldf.to_string())
paneldf_v2.head(10)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name
Element ID,,,,,
1,Subcase 1 (LC1),-65.78,25.90,12.58,panel1
2,Subcase 1 (LC1),-81.87,26.74,20.59,panel1
3,Subcase 1 (LC1),-66.73,36.48,20.47,panel1
4,Subcase 1 (LC1),-96.38,34.06,2.38,panel2
5,Subcase 1 (LC1),-88.32,26.68,25.01,panel2
6,Subcase 1 (LC1),-95.01,28.08,26.25,panel2
7,Subcase 1 (LC1),-92.89,33.03,-2.92,panel3
8,Subcase 1 (LC1),-94.89,26.44,19.75,panel3
9,Subcase 1 (LC1),-92.33,29.37,36.81,panel3


In [ ]:
# Import the stress of the stringers 
stringerdf = pd.read_excel(f'../data/{name}/stringer.xlsx', index_col=0)

stringerdf_v2 = pd.read_csv(f'../data/{name}/stringer_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

stringerdf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)

# now add the component column to the paneldf_v2 by matching the element_id
for index, row in stringerdf_v2.iterrows():
    element_id = row['Element ID']
    # Find the matching row in the element_component_df
    matching_row = element_component_df[element_component_df['Element ID'] == element_id]
    if not matching_row.empty:
        component = matching_row['Component Name'].values[0]
        stringerdf_v2.at[index, 'Component Name'] = component
    else:
        stringerdf_v2.at[index, 'Component Name'] = None  # or handle missing components as needed

stringerdf_v2.set_index('Element ID', inplace=True)

stringerdf_v2.rename(columns={'Element Stresses (1D):CBAR/CBEAM Axial Stress': 'sigmaXX', 'Loadcase': 'Load Case'}, inplace=True)
stringerdf_v2.drop(columns=['FileID', 'Step'], inplace=True)

stringerdf_v2['Load Case'] = stringerdf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

#print(stringerdf.head(30))
stringerdf_v2.head(10)


,Load Case,sigmaXX,Component Name
Element ID,,,
40,Subcase 1 (LC1),-76.64,stringer1
41,Subcase 1 (LC1),-69.67,stringer1
42,Subcase 1 (LC1),-78.63,stringer1
43,Subcase 1 (LC1),-80.22,stringer2
44,Subcase 1 (LC1),-84.12,stringer2
45,Subcase 1 (LC1),-84.03,stringer2
46,Subcase 1 (LC1),-80.34,stringer3
47,Subcase 1 (LC1),-82.80,stringer3
48,Subcase 1 (LC1),-84.04,stringer3


## Constants 

In [ ]:
sigma_ult = 530

## Applying Functions 

In [ ]:
#Adding reserve Factor column to the panels 
paneldf['reserveFactor'] = paneldf.apply(panelStrength_calc, sigma_ult= sigma_ult, axis=1)
paneldf.head(10)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name,reserveFactor
Element ID,,,,,,
1,Subcase 1 (LC1),-65.78,25.90,12.58,panel1,4.128332
2,Subcase 1 (LC1),-81.87,26.74,20.59,panel1,3.375439
3,Subcase 1 (LC1),-66.73,36.48,20.47,panel1,3.493345
4,Subcase 1 (LC1),-96.38,34.06,2.38,panel2,3.098414
5,Subcase 1 (LC1),-88.32,26.68,25.01,panel2,3.126688
6,Subcase 1 (LC1),-95.01,28.08,26.25,panel2,2.926671
7,Subcase 1 (LC1),-92.89,33.03,-2.92,panel3,3.275147
8,Subcase 1 (LC1),-94.89,26.44,19.75,panel3,3.056275
9,Subcase 1 (LC1),-92.33,29.37,36.81,panel3,2.805093


In [ ]:
#Adding reserve Factor column to the stringers 
stringerdf['reserveFactor'] = stringerdf.apply(stringerStrength_calc, sigma_ult= sigma_ult, axis=1)
stringerdf.head(10)

,Component Name,sigmaXX,Load Case,reserveFactor
Element ID,,,,
40,stringer1,-76.64,Subcase 1 (LC1),4.610299
41,stringer1,-69.67,Subcase 1 (LC1),5.071528
42,stringer1,-78.63,Subcase 1 (LC1),4.493620
43,stringer2,-80.22,Subcase 1 (LC1),4.404554
44,stringer2,-84.12,Subcase 1 (LC1),4.200349
45,stringer2,-84.03,Subcase 1 (LC1),4.204847
46,stringer3,-80.34,Subcase 1 (LC1),4.397975
47,stringer3,-82.80,Subcase 1 (LC1),4.267311
48,stringer3,-84.04,Subcase 1 (LC1),4.204347


# Making things pretty 

## Delete unnecessary columns 

In [ ]:
# First for the stringers 
stringerdf = stringerdf.drop(['Component Name','sigmaXX'], axis=1)
stringerdf

,Load Case,reserveFactor
Element ID,,
40,Subcase 1 (LC1),4.610299
41,Subcase 1 (LC1),5.071528
42,Subcase 1 (LC1),4.493620
43,Subcase 1 (LC1),4.404554
44,Subcase 1 (LC1),4.200349
...,...,...
62,Subcase 3 (LC3),12.646147
63,Subcase 3 (LC3),17.944811
64,Subcase 3 (LC3),11.880744


In [ ]:
# Then also the panels 
paneldf=paneldf.drop(['Component Name', 'sigmaXX', 'sigmaYY', 'sigmaXY'], axis = 1)
paneldf.head(10)

,Load Case,reserveFactor
Element ID,,
1,Subcase 1 (LC1),4.128332
2,Subcase 1 (LC1),3.375439
3,Subcase 1 (LC1),3.493345
4,Subcase 1 (LC1),3.098414
5,Subcase 1 (LC1),3.126688
6,Subcase 1 (LC1),2.926671
7,Subcase 1 (LC1),3.275147
8,Subcase 1 (LC1),3.056275
9,Subcase 1 (LC1),2.805093


## Concat data frames beneth each other 

In [ ]:
combined = pd.concat([paneldf,stringerdf],axis=0)
combined.sample(20)

,Load Case,reserveFactor
Element ID,,
5,Subcase 1 (LC1),3.126688
63,Subcase 1 (LC1),4.404554
42,Subcase 2 (LC2),4.120505
59,Subcase 2 (LC2),4.527593
59,Subcase 1 (LC1),4.267311
4,Subcase 3 (LC3),1.607326
15,Subcase 2 (LC2),2.562813
60,Subcase 3 (LC3),17.728717
61,Subcase 3 (LC3),10.129969


## Split into three seperate loadcase data frames 

In [ ]:
loadCase1df = combined[combined['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = combined[combined['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = combined[combined['Load Case'] == 'Subcase 3 (LC3)']
loadCase1df.head(20)

,Load Case,reserveFactor
Element ID,,
1,Subcase 1 (LC1),4.128332
2,Subcase 1 (LC1),3.375439
3,Subcase 1 (LC1),3.493345
4,Subcase 1 (LC1),3.098414
5,Subcase 1 (LC1),3.126688
6,Subcase 1 (LC1),2.926671
7,Subcase 1 (LC1),3.275147
8,Subcase 1 (LC1),3.056275
9,Subcase 1 (LC1),2.805093


In [ ]:
# Drop the loadcase columns 
loadCase1df = loadCase1df.drop(['Load Case'], axis = 1)
loadCase2df = loadCase2df.drop(['Load Case'], axis = 1)
loadCase3df = loadCase3df.drop(['Load Case'], axis = 1)
#Rename the reserveFactor column 
loadCase1df = loadCase1df.rename(columns={"reserveFactor": "Load Case 1 RF"})
loadCase2df = loadCase2df.rename(columns={"reserveFactor": "Load Case 2 RF"})
loadCase3df = loadCase3df.rename(columns={"reserveFactor": "Load Case 3 RF"})
loadCase2df.tail(5)

,Load Case 2 RF
Element ID,
62,4.394693
63,4.625387
64,4.120505
65,4.016521
66,4.217898


In [ ]:
# Now we concat them back together 
outputdf = pd.concat([loadCase1df,loadCase2df, loadCase3df], axis=1)
outputdf

,Load Case 1 RF,Load Case 2 RF,Load Case 3 RF
Element ID,,,
1,4.128332,4.636545,1.658619
2,3.375439,3.979656,1.887111
3,3.493345,4.039993,1.592438
4,3.098414,3.472703,1.607326
5,3.126688,3.759188,1.885314
6,2.926671,3.408633,1.573843
7,3.275147,3.560446,1.565454
8,3.056275,3.555657,1.900334
9,2.805093,3.310910,1.485860


# ROUND & Export back to Excel 

In [ ]:
outputdf = outputdf.round(rounding_digits)
outputdf.to_excel(f'../data/{name}/output/processed_d.xlsx')
outputdf.head(10)

,Load Case 1 RF,Load Case 2 RF,Load Case 3 RF
Element ID,,,
1,4.13,4.64,1.66
2,3.38,3.98,1.89
3,3.49,4.04,1.59
4,3.10,3.47,1.61
5,3.13,3.76,1.89
6,2.93,3.41,1.57
7,3.28,3.56,1.57
8,3.06,3.56,1.90
9,2.81,3.31,1.49
